<a href="https://colab.research.google.com/github/michalis0/BigScaleAnalytics/blob/master/week3/2-restaurants_deduplication.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# if you don't have recordlinkage installed
!pip install recordlinkage

     |████████████████████████████████| 944 kB 5.2 MB/s 
     |████████████████████████████████| 132 kB 32.3 MB/s 
  Created wheel for jellyfish: filename=jellyfish-0.9.0-cp37-cp37m-linux_x86_64.whl size=73968 sha256=df767dba9706228f31159bd3d5a6ca1c091225b108f4d724341be869e86c55ee
  Stored in directory: /root/.cache/pip/wheels/fe/99/4e/646ce766df0d070b0ef04db27aa11543e2767fda3075aec31b
Successfully built jellyfish


In [2]:
import pandas as pd
import recordlinkage

In [4]:
df = pd.read_csv("https://raw.githubusercontent.com/michalis0/BigScaleAnalytics/master/week3/data/restaurants.csv", delimiter=",")
targets = df["class"]
df = df.iloc[:, :-1].set_index(keys="id")
df.head(10)

,name,addr,city,phone,type
id,,,,,
1,arnie morton's of chicago,435 s. la cienega blv.,los angeles,310/246-1501,american
2,arnie morton's of chicago,435 s. la cienega blvd.,los angeles,310-246-1501,steakhouses
3,art's delicatessen,12224 ventura blvd.,studio city,818/762-1221,american
4,art's deli,12224 ventura blvd.,studio city,818-762-1221,delis
5,hotel bel-air,701 stone canyon rd.,bel air,310/472-1211,californian
6,bel-air hotel,701 stone canyon rd.,bel air,310-472-1211,californian
7,cafe bizou,14016 ventura blvd.,sherman oaks,818/788-3536,french
8,cafe bizou,14016 ventura blvd.,sherman oaks,818-788-3536,french bistro
9,campanile,624 s. la brea ave.,los angeles,213/938-1447,american


### Data cleaning
It is important to clean up a bit the data before jumping into finding duplicates. This not only helps to find matches more easily but also makes our blocking index to work better. Examples of data cleaning steps include standardising the order of first and surname, standardising the addresses dates and phone numbers, lowercasing all of the characters, etc. 

As you already know, Pandas itself is very useful for data cleaning, however Record Linkage Toolkit has also some cleaning function. We are going to use `recordlinkage.preprocessing.clean()` which is the most generic function in this package.

In [5]:
# cleaning the name column
from recordlinkage.preprocessing import clean
df["name"] = clean(df["name"], lowercase=True, strip_accents='unicode', remove_brackets=True)
# addr
df["addr"] = clean(df["addr"], lowercase=True, strip_accents='unicode', remove_brackets=True)
# city
df["city"] = clean(df["city"], lowercase=True, strip_accents='unicode', remove_brackets=True)
# type
df["type"] = clean(df["type"], lowercase=True, strip_accents='unicode', remove_brackets=True)

/usr/local/lib/python3.7/dist-packages/recordlinkage/preprocessing/cleaning.py:114: FutureWarning: The default value of regex will change from True to False in a future version.
  s = s.str.replace(r'(\[.*?\]|\(.*?\)|\{.*?\})', '')
/usr/local/lib/python3.7/dist-packages/recordlinkage/preprocessing/cleaning.py:118: FutureWarning: The default value of regex will change from True to False in a future version.
  s = s.str.replace(replace_by_none, '')
/usr/local/lib/python3.7/dist-packages/recordlinkage/preprocessing/cleaning.py:121: FutureWarning: The default value of regex will change from True to False in a future version.
  s = s.str.replace(replace_by_whitespace, ' ')
/usr/local/lib/python3.7/dist-packages/recordlinkage/preprocessing/cleaning.py:124: FutureWarning: The default value of regex will change from True to False in a future version.
  s = s.str.replace(r'\s\s+', ' ')


In [6]:
# there are also more specific cleaning functions in this package
from recordlinkage.preprocessing import phonenumbers
df["phone"] = phonenumbers(df["phone"])

/usr/local/lib/python3.7/dist-packages/recordlinkage/preprocessing/cleaning.py:148: FutureWarning: The default value of regex will change from True to False in a future version.
  s = s.astype(object).str.replace('[^0-9+]+', '')


In [7]:
df.head(10)

,name,addr,city,phone,type
id,,,,,
1,arnie mortons of chicago,435 s la cienega blv,los angeles,3102461501,american
2,arnie mortons of chicago,435 s la cienega blvd,los angeles,3102461501,steakhouses
3,arts delicatessen,12224 ventura blvd,studio city,8187621221,american
4,arts deli,12224 ventura blvd,studio city,8187621221,delis
5,hotel bel air,701 stone canyon rd,bel air,3104721211,californian
6,bel air hotel,701 stone canyon rd,bel air,3104721211,californian
7,cafe bizou,14016 ventura blvd,sherman oaks,8187883536,french
8,cafe bizou,14016 ventura blvd,sherman oaks,8187883536,french bistro
9,campanile,624 s la brea ave,los angeles,2139381447,american


Now let's choose a blocking index for this data set to make record pairs. We will pick the `city` attribute as the blocking index, since it seems that this is the feature which may not change for a restaurant.

In [8]:
indexer = recordlinkage.Index()
indexer.block('city')
candidate_links = indexer.index(df)

print(len(candidate_links))

57943


In [9]:
compare_cl = recordlinkage.Compare()

# exact comparison for city since it is the blocking index
compare_cl.exact('city', 'city', label='city')
# partial similarity between string values
compare_cl.string('name', 'name', method='jarowinkler', threshold=0.85, label='surname')
compare_cl.string('addr', 'addr', threshold=0.85, label='addr')
# phone is not really a numerical value, therefore it's better to use a string comparison
compare_cl.string('phone', 'phone', threshold=0.85, label='phone')
compare_cl.string('type', 'type', method='jarowinkler', threshold=0.85, label='type')

features = compare_cl.compute(candidate_links, df)

In [10]:
features.head(20)

city  surname  addr  phone  type
id_1 id_2                                  
2    1        1      1.0   1.0    1.0   0.0
9    1        1      0.0   0.0    0.0   1.0
     2        1      0.0   0.0    0.0   0.0
10   1        1      0.0   0.0    0.0   0.0
     2        1      0.0   0.0    0.0   0.0
     9        1      1.0   1.0    1.0   0.0
13   1        1      0.0   0.0    0.0   0.0
     2        1      0.0   0.0    0.0   0.0
     9        1      0.0   0.0    0.0   0.0
     10       1      0.0   0.0    0.0   1.0
14   1        1      0.0   0.0    0.0   0.0
     2        1      0.0   0.0    0.0   0.0
     9        1      0.0   0.0    0.0   0.0
     10       1      0.0   0.0    0.0   1.0
     13       1      1.0   1.0    1.0   1.0
19   1        1      0.0   0.0    0.0   1.0
     2        1      0.0   0.0    0.0   0.0
     9        1      0.0   0.0    0.0   1.0
     10       1      0.0   0.0    0.0   0.0
     13       1      0.0   0.0    0.0   0.0

### Classification
In general there are three ways to classify the candidate record pairs as matches or non-matches:

- Threshold-based methods
- Supervised learning methods
- Unsupervised learning methods

Once we classify each of the record pairs, we can evaluate the classification using three commonly used metrics: precision, recall and F-score.


#### Threshold based

In [11]:
# Sum the comparison results.
features.sum(axis=1).value_counts().sort_index(ascending=False)

5.0       21
4.0       29
3.0       92
2.0     8288
1.0    49513
dtype: int64

In [12]:
matches = features[features.sum(axis=1) >= 4]

print(len(matches))
matches.head(10)

50


,,city,surname,addr,phone,type
id_1,id_2,,,,,
2,1,1,1.0,1.0,1.0,0.0
10,9,1,1.0,1.0,1.0,0.0
14,13,1,1.0,1.0,1.0,1.0
26,25,1,1.0,1.0,1.0,1.0
28,27,1,1.0,0.0,1.0,1.0
34,33,1,1.0,1.0,1.0,1.0
40,39,1,1.0,1.0,1.0,0.0
4,3,1,1.0,1.0,1.0,0.0
6,5,1,0.0,1.0,1.0,1.0


##### precision and recall

In [13]:
# the restaurants gold data contains the true matches between data pairs
true_links = pd.read_csv("https://raw.githubusercontent.com/michalis0/BigScaleAnalytics/master/week3/data/restaurant_gold.csv")
true_links.head()

,class,id_1,id_2
0,'0',1,2
1,'1',3,4
2,'2',5,6
3,'3',7,8
4,'4',9,10


In [14]:
# make sure the order of the indices match. here, in the matches dataframe the first index is
# always the bigger number whears in the true links dataframe it is reverse. So we have to set
# the multi-index for true links dataframe in the order of `id_2`, `id_1`
true_links = true_links.set_index(keys=["id_2", "id_1"])

In [15]:
print("precision = ", recordlinkage.precision(true_links, matches))

precision =  0.94


/usr/local/lib/python3.7/dist-packages/recordlinkage/measures.py:147: FutureWarning: Index.__and__ operating as a set operation is deprecated, in the future this will be a logical operation matching Series.__and__.  Use index.intersection(other) instead
  return len(links_true & links_pred)


In [16]:
print("recall = ", recordlinkage.recall(true_links, matches))

recall =  0.41964285714285715


/usr/local/lib/python3.7/dist-packages/recordlinkage/measures.py:147: FutureWarning: Index.__and__ operating as a set operation is deprecated, in the future this will be a logical operation matching Series.__and__.  Use index.intersection(other) instead
  return len(links_true & links_pred)


In [17]:
print("F-score = ", recordlinkage.fscore(true_links, matches))

F-score =  0.5802469135802469


/usr/local/lib/python3.7/dist-packages/recordlinkage/measures.py:147: FutureWarning: Index.__and__ operating as a set operation is deprecated, in the future this will be a logical operation matching Series.__and__.  Use index.intersection(other) instead
  return len(links_true & links_pred)


#### Supervised learning
In this part we will use Logistic Regression to classify the candidate pairs.

In [18]:
from recordlinkage.classifiers import LogisticRegressionClassifier
LR = LogisticRegressionClassifier()

Let's first add labels to the feature vectors we created. This way we can have balanced number of actual matches and non-matches in the train and test set.

In [19]:
join_feature_label = features.reset_index().join(true_links, how='left', on=["id_1", "id_2"])
join_feature_label["class"] = join_feature_label["class"].map(lambda x: 0 if pd.isnull(x) else 1)

In [20]:
join_feature_label.set_index(keys=["id_1", "id_2"], inplace=True)
join_feature_label.head(20)

city  surname  addr  phone  type  class
id_1 id_2                                         
2    1        1      1.0   1.0    1.0   0.0      1
9    1        1      0.0   0.0    0.0   1.0      0
     2        1      0.0   0.0    0.0   0.0      0
10   1        1      0.0   0.0    0.0   0.0      0
     2        1      0.0   0.0    0.0   0.0      0
     9        1      1.0   1.0    1.0   0.0      1
13   1        1      0.0   0.0    0.0   0.0      0
     2        1      0.0   0.0    0.0   0.0      0
     9        1      0.0   0.0    0.0   0.0      0
     10       1      0.0   0.0    0.0   1.0      0
14   1        1      0.0   0.0    0.0   0.0      0
     2        1      0.0   0.0    0.0   0.0      0
     9        1      0.0   0.0    0.0   0.0      0
     10       1      0.0   0.0    0.0   1.0      0
     13       1      1.0   1.0    1.0   1.0      1
19   1        1      0.0   0.0    0.0   1.0      0
     2        1      0.0   0.0    0.0   0.0      0
     9        1      0.0   0.0    0.0   1.0      0
     10       1      0.0   0.0    0.0   0.0      0
     13       1      0.0   0.0    0.0   0.0      0

In [21]:
# train-test split
from sklearn.model_selection import train_test_split
train, test, train_labels, test_labels = train_test_split(join_feature_label.iloc[:, :-1], 
                                                          join_feature_label["class"],
                                                          test_size=0.3)

In [22]:
# find the indices in the train data which contain a true match
train_matches_index = train.index & true_links.index

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: FutureWarning: Index.__and__ operating as a set operation is deprecated, in the future this will be a logical operation matching Series.__and__.  Use index.intersection(other) instead
  


In [23]:
train_matches_index

MultiIndex([( 18,  17),
            (202, 201),
            (154, 153),
            (170, 169),
            (206, 205),
            (198, 197),
            (210, 209),
            ( 14,  13),
            (142, 141),
            (208, 207),
            (156, 155),
            (180, 179),
            (178, 177),
            (212, 211),
            (222, 221),
            (158, 157),
            (190, 189),
            ( 30,  29),
            (  4,   3),
            (192, 191),
            (182, 181),
            (194, 193),
            ( 40,  39),
            (176, 175),
            (174, 173),
            ( 26,  25),
            (160, 159),
            (162, 161),
            (196, 195),
            (150, 149),
            (172, 171),
            ( 28,  27),
            (138, 137),
            (148, 147),
            ( 12,  11),
            (216, 215),
            (  2,   1),
            (184, 183),
            (188, 187),
            (220, 219),
            (204, 203),
            (146

In [24]:
# find the indices in the test data containing a true match
test_matches_index = test.index & true_links.index

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: FutureWarning: Index.__and__ operating as a set operation is deprecated, in the future this will be a logical operation matching Series.__and__.  Use index.intersection(other) instead
  


In [25]:
_ = LR.fit_predict(train, train_matches_index)

/usr/local/lib/python3.7/dist-packages/recordlinkage/base.py:950: FutureWarning: Index.__and__ operating as a set operation is deprecated, in the future this will be a logical operation matching Series.__and__.  Use index.intersection(other) instead
  y.loc[match_index & comparison_vectors.index] = 1


In [26]:
LR_test_matches = LR.predict(test)

##### precision and recall

In [27]:
print("precision = ",recordlinkage.precision(test_matches_index, LR_test_matches))

precision =  1.0


/usr/local/lib/python3.7/dist-packages/recordlinkage/measures.py:147: FutureWarning: Index.__and__ operating as a set operation is deprecated, in the future this will be a logical operation matching Series.__and__.  Use index.intersection(other) instead
  return len(links_true & links_pred)


In [28]:
print("recall = ", recordlinkage.recall(test_matches_index, LR_test_matches))

recall =  0.8333333333333334


/usr/local/lib/python3.7/dist-packages/recordlinkage/measures.py:147: FutureWarning: Index.__and__ operating as a set operation is deprecated, in the future this will be a logical operation matching Series.__and__.  Use index.intersection(other) instead
  return len(links_true & links_pred)


In [29]:
print("F-score = ", recordlinkage.fscore(test_matches_index, LR_test_matches))

F-score =  0.9090909090909091


/usr/local/lib/python3.7/dist-packages/recordlinkage/measures.py:147: FutureWarning: Index.__and__ operating as a set operation is deprecated, in the future this will be a logical operation matching Series.__and__.  Use index.intersection(other) instead
  return len(links_true & links_pred)


We can observe a significant improvment in the supervised learning method compared to threshold based method.

#### Unsupervised learning
We can also use unsupervised methods to classify candidate pairs. Here we are going to use [KMeans](https://en.wikipedia.org/wiki/K-means_clustering). KMeans partiotions candidate record pairs into matches and non-matches. Each comparison vector belongs to the clustcer with the nearest mean. The algorithm is calibrated for two clusters: a match cluster and a non-match cluster). The centers of these clusters can be given as arguments or set automatically.

In [30]:
kmeans = recordlinkage.KMeansClassifier()

let's use the same train and test set as we used for logistic regression. Note that here there is no training involved and we are doing the splitting to see how does our model perform in case of new data coming in.

In [31]:
kmeans.fit_predict(train)

kmeans_test_matches = kmeans.predict(test)

##### precision and recall

In [32]:
print("precision = ",recordlinkage.precision(test_matches_index, kmeans_test_matches))

/usr/local/lib/python3.7/dist-packages/recordlinkage/measures.py:147: FutureWarning: Index.__and__ operating as a set operation is deprecated, in the future this will be a logical operation matching Series.__and__.  Use index.intersection(other) instead
  return len(links_true & links_pred)


precision =  0.3333333333333333


In [33]:
print("recall = ", recordlinkage.recall(test_matches_index, kmeans_test_matches))

recall =  1.0


/usr/local/lib/python3.7/dist-packages/recordlinkage/measures.py:147: FutureWarning: Index.__and__ operating as a set operation is deprecated, in the future this will be a logical operation matching Series.__and__.  Use index.intersection(other) instead
  return len(links_true & links_pred)


In [34]:
print("F-score = ", recordlinkage.fscore(test_matches_index, kmeans_test_matches))

F-score =  0.5


/usr/local/lib/python3.7/dist-packages/recordlinkage/measures.py:147: FutureWarning: Index.__and__ operating as a set operation is deprecated, in the future this will be a logical operation matching Series.__and__.  Use index.intersection(other) instead
  return len(links_true & links_pred)
